In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

np.random.seed(0)

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
# df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = pd.read_csv("/data/funds.csv")

In [3]:
# df.to_csv("/data/funds.csv", index=False)

In [4]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID108406,Maile Treston,Charles Darwin University,Male,Lakes Entrance,Victoria,Y,FID000044,NOT APPLICABLE,-37.8667,...,52,Senior,13,MORE_THAN_10_YEAR,3,2.0,1972-04-29,2011-02-25,9999-12-31,2037-04-29
1,MID108415,Annabelle Cryer,RMIT University,Female,Mooroopna,Victoria,N,FID000016,NOT APPLICABLE,-36.3833,...,26,Young,39,MORE_THAN_10_YEAR,3,2.0,1998-10-05,2010-01-04,9999-12-31,2063-10-05
2,MID108429,Leonora Acuff,University of South Australia,Male,Jindabyne,New South Wales,N,FID000100,NOT APPLICABLE,-36.4000,...,51,Senior,14,MORE_THAN_10_YEAR,3,2.0,1973-12-20,2010-09-04,9999-12-31,2038-12-20
3,MID108436,Franklyn Gesick,The University of Newcastle,Male,Wingham,New South Wales,N,FID000012,NOT APPLICABLE,-31.8667,...,41,Middle-aged,24,MORE_THAN_10_YEAR,3,2.0,1983-03-14,2013-04-27,9999-12-31,2048-03-14
4,MID108438,Amber Gillaspie,University of South Australia,Female,Nathalia,Victoria,Y,FID000035,NOT APPLICABLE,-36.0583,...,40,Middle-aged,25,MORE_THAN_10_YEAR,3,2.0,1984-01-06,2012-05-17,9999-12-31,2049-01-06


In [5]:
df.isna().sum() / df.shape[0]

MEMBER_ID                                           0.000000
MEMBER_NAME                                         0.000000
MEMBER_EMPLOYMENT                                   0.000000
MEMBER_GENDER                                       0.000000
MEMBER_CITY_TOWN                                    0.000000
MEMBER_STATE                                        0.000000
MEMBER_CONTACT_VERIFIED                             0.000000
FUND_ID                                             0.000000
CHURN_REASON                                        0.000000
LATITUDE                                            0.000000
LONGITUDE                                           0.000000
COUNTRY                                             0.000000
COUNTRYCODE                                         0.000000
CAPITAL                                             0.000000
POPULATION                                          0.000000
FUND_TOTAL_ASSETS                                   0.000000
FUND_RETURN_TARGET_PERCE

NO_OF_CHURN column has almost 50% missing value, it wont be wise to impute these many values so will just drop it.                                     

In [6]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [7]:
frame["CHURN_FLAG"] = frame["CHURN_FLAG"].apply(lambda x: 1 if x =="Y" else 0)

In [82]:
# frame.groupby(["RETIREMENT_AGE_GROUP","CHURN_FLAG"])[["CHURN_FLAG"]].count()

The distribution of CHURN is pretty common bettween the employers and hence wont contribute inn modelling

In [76]:
# multiple = {i for i, j in dict(df["MEMBER_ID"].value_counts()).items() if j > 1 }
# multiple[multiple["value_counts"]>=2].index.to_list()

# MODEL TO PREDICT CHURN

In [8]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [9]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(n_estimators=500, max_depth=8, 
                                                                                 max_features=0.6,
                                                                                 bootstrap=True, max_samples=0.8))]
)



In [10]:
X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [11]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.984


In [12]:
import pickle

# save
with open('churn_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf,f)

In [13]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['FUND_TOTAL_ASSETS',
                                                   'FUND_RETURN_TARGET_PERCENTAGE',
                                                   'CASH_BENCHMARK_ALLOCATION',
                                                   'FIXED_INCOME_BENCHMARK_ALLOCATION',
                                                   'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
                                                   'INTERNATIONAL_LISTED_EQU...
                                                                                    score_func=<function chi2 at 0x7f0dc09f3040>))]),
                                                  ['MEMBER_GENDER',
                                                   'MEMBER_STATE',
                                                   'MEMBER_CONTACT_VERIFIED',
                                                   'INVESTMENT_RISK_CATEGORY',
                                                   'FUND_RISK_LEVEL',
                                                   'FUND_RISK_CATEGORY',
                                                   'NEG_NETRETURN_SINCE_INCEPTION',
                                                   'INVESTMENT_AGE_GROUP',
                                                   'RETIREMENT_AGE_GROUP',
                                                   'TOTAL_FUNDS_INVESTED'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=8, max_features=0.6,
                                        max_samples=0.8, n_estimators=500))])

In [ ]:
frame = clf.predict(frame)

In [ ]:
df["CHURN_PREDICTED"] = frame

In [ ]:
churned = df[df["CHURN_FLAG"] == "Y"]

In [ ]:
churned.groupby(["CHURN_REASON"])[["CHURN_REASON"]].count() / churned.shape[0]*100

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
frame = churned[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED',"CHURN_REASON"]].copy()

In [ ]:
churn_reason_dic = {reason:i for i, reason in enumerate(churned["CHURN_REASON"].unique())}
frame["CHURN_REASON"] = frame["CHURN_REASON"].apply(lambda x: churn_reason_dic[x])

In [ ]:
frame

In [ ]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [ ]:
multi_lable_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=500, max_depth=8, max_features=0.6, bootstrap=True,
                                                             max_samples=0.8))
clf2 = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", multi_lable_clf)]
)




In [ ]:
X = frame.drop("CHURN_REASON", axis=1)
y = frame["CHURN_REASON"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_test, y_test))

In [ ]:
import pickle

# save
with open('reason_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf2,f)

# FUND RECOMMENDATION

In [27]:
df.columns

Index(['MEMBER_ID', 'MEMBER_NAME', 'MEMBER_EMPLOYMENT', 'MEMBER_GENDER',
       'MEMBER_CITY_TOWN', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED',
       'FUND_ID', 'CHURN_REASON', 'LATITUDE', 'LONGITUDE', 'COUNTRY',
       'COUNTRYCODE', 'CAPITAL', 'POPULATION', 'FUND_TOTAL_ASSETS',
       'FUND_RETURN_TARGET_PERCENTAGE', 'INVESTMENT_RISK_LEVEL',
       'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION',
       'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION',
       'PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION',
       'FUND_RISK_LEVEL', 'FUND_RISK_CATEGORY',
       'NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES',
       'PEN

In [28]:
df[["FUND_ID","CHURN_REASON"]]

,FUND_ID,CHURN_REASON
0,FID000044,NOT APPLICABLE
1,FID000016,NOT APPLICABLE
2,FID000100,NOT APPLICABLE
3,FID000012,NOT APPLICABLE
4,FID000035,NOT APPLICABLE
...,...,...
335994,FID000119,NOT APPLICABLE
335995,FID000026,FUND REPUTATION DECLINING
335996,FID000017,NO REASON IDENTIFIED
335997,FID000191,FUND REPUTATION DECLINING


In [38]:
fund_allocation = session.table("FUND_ALLOCATION_MASTER").to_pandas()
fund_master = session.table("FUND_MASTER").to_pandas()
fund_performance = session.table("FUND_PERFORMANCE_MASTER").to_pandas()

In [39]:
fund = pd.merge(fund_allocation, fund_master, on="FUND_ID")
fund_total = pd.merge(fund, fund_performance, on="FUND_ID")


In [40]:
fund_meta = fund_total[["FUND_ID","FUNDNAME"]]
fund_cluster = fund_total.drop(["FUND_ID","FUNDNAME","FUND_NAME_y","FUND_TRUSTEE","FUND_INCEPTION_DATE"], axis=1)

In [44]:
from sklearn.cluster import DBSCAN

# X = np.array([[1, 2], [2, 2], [2, 3],
#               [8, 7], [8, 8], [25, 80]])
# clustering = DBSCAN(eps=3, min_samples=2).fit(X)
# clustering.labels_

In [45]:
fund_data = pd.get_dummies(fund_cluster)

In [50]:
fund_data = fund_data.replace(False, 0)
fund_data = fund_data.replace(True, 1)

In [59]:
clustering = DBSCAN(eps=2, min_samples=4).fit(fund_data)
# clustering.labels_